<a href="https://colab.research.google.com/github/NamyaVyas/002-random-forest-example/blob/main/ssp_tcn_200M_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import h5py

from keras.utils import to_categorical
import copy

import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

Using TensorFlow backend.
/home/hegder/anaconda3/envs/dlc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hegder/anaconda3/envs/dlc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hegder/anaconda3/envs/dlc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hegder/anaconda3/envs/d

In [ ]:
# Min-Max scaling of input

xxx = np.loadtxt('XXX_200M.txt')
np.shape(xxx)

(200001969, 10)

In [ ]:
xxx = np.array(xxx)/255

In [ ]:
# s_tr = 100000000
# x_tr = copy.deepcopy(xxx[:s_tr])

In [ ]:
s_tr = 100000000
x_te = copy.deepcopy(xxx[s_tr:])

In [ ]:
del xxx

In [ ]:
# Read the file
ff1 = open ('m_32_200M.txt', 'r')
ff2 = ff1.read()

In [ ]:
# Divide the bitstream, xx will be bitstream, yy will be decimal integer of bitstream

span = 10
xy_l = (span+1)*8
xy = []
for i in range (len(ff2)-xy_l+1):
    xy.append(ff2[i:i+xy_l]) 

yy = []
for i in range(100000000,200000000):
    yy.append(int(xy[i][span*8:],2))
del(xy)
del(ff1)
del(ff2)

In [ ]:
# # Encode the input xx in decimal
# # xxx = []
# for i in range(len(xx)):
#     temp = []
#     aa = np.reshape(list(xx[i]), (span,8))
#     for j in range (span):
#         temp.append(int("".join(aa[j]), 2))
#     with open('XXX_200M.txt','a') as xxx:
#         np.savetxt(xxx, [temp], delimiter='\t',fmt='%s')

In [ ]:
# Encode the output yy in one-hot
yyy = to_categorical(yy, num_classes = 256) 

In [ ]:
del yy

In [ ]:
s_tr

100000000

In [ ]:
# for i in range (len(yyy)):
#     with open('YYY_200M.txt','a') as yyy1:
#         np.savetxt(yyy1, [yyy[i]], delimiter='\t',fmt='%i')

In [ ]:
import sys
sys.getsizeof(yyy)

112

In [ ]:
# y_tr = copy.deepcopy(yyy[:s_tr])
y_te = copy.deepcopy(yyy)

In [ ]:
np.shape(y_te)

(100000000, 256)

In [ ]:
del yyy

In [ ]:
# Train-test split
s_tr = 100000000
# x_tr, x_te, y_tr, y_te = xxx[:s_tr], xxx[s_tr:1000000], yyy[:s_tr], yyy[s_tr:1000000]
x_tr, x_te, y_tr, y_te = copy.deepcopy(xxx[:s_tr]), copy.deepcopy(xxx[s_tr:]), copy.deepcopy(yyy[:s_tr]), copy.deepcopy(yyy[s_tr:])

# x_tr, x_te, y_tr, y_te = xxx, xxx1, yyy, yyy1

In [ ]:
np.shape(x_te)

(100001969, 10, 1)

In [ ]:
# x_tr = np.reshape(x_tr, (x_tr.shape[0], x_tr.shape[1], 1))
x_te = np.reshape(x_te, (x_te.shape[0], x_te.shape[1], 1))

In [ ]:
# Second regressor model  TCN based 
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense,Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
# if time_steps > tcn_layer.receptive_field, then we should not
# be able to solve this task.
batch_size, time_steps, input_dim = None, x_tr.shape[1],1
tcn_layer = TCN(input_shape=(time_steps, input_dim), 
                kernel_size=7, 
                dilations=(1, 2, 4, 8, 16, 32,64), 
                dropout_rate=0.2,
                use_batch_norm=False)
# The receptive field tells you how far the model can see in terms of timesteps.
print('Receptive field size =', tcn_layer.receptive_field)

regressor2 = Sequential()
regressor2.add(tcn_layer)       
#regressor2.add(Dense(units = 32, activation='relu'     ))
regressor2.add(Dense(256, activation='relu'))
regressor2.add(Dense(256, activation='softmax'))

regressor2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

regressor2.summary()
#tcn_full_summary(regressor2, expand_residual_blocks=False)
regressor2.fit(x_tr, yyy, epochs=50, batch_size=1024)

In [ ]:
regressor2.save('/media/DiskC/qrng/ssp_tcn_200M')

In [ ]:
regressor2.save_weights('/media/DiskC/qrng/weights_ssp_tcn_200M')

In [ ]:
from tcn import TCN
model = tf.keras.models.load_model('/media/DiskC/qrng/ssp_tcn_200M', custom_objects={'TCN': TCN})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
x_te = np.reshape(x_te, (x_te.shape[0], x_te.shape[1], 1))
y_pre = model.predict(x_te)

y_pre1 = np.argmax(y_pre, axis=1)
y_te1 = np.argmax(y_te, axis=1)

score = sum(y_pre1 == y_te1) / len(y_te1)
print(score)

/home/hegder/anaconda3/envs/dlc/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys


TypeError: 'bool' object is not iterable

In [ ]:
score = sum(int(y_pre1 == y_te1)) / len(y_te1)
print(score)

/home/hegder/anaconda3/envs/dlc/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


TypeError: 'int' object is not iterable

In [ ]:
np.shape(y_pre1)
# y_te[1]

(100001969,)

In [ ]:
sj = np.array([[1,2,3,5,4,8,6,4,65,2],[1,5,8,9,6],[9]])
sj
# np.shape(sj)


array([list([1, 2, 3, 5, 4, 8, 6, 4, 65, 2]), list([1, 5, 8, 9, 6]),
       list([9])], dtype=object)

In [ ]:
# 20-100%
# 22-99.43%
# 24-99.73%
# 26-38.9%
#28-40.25%
# 30-44%


#26-dialation=32-34s - 23.91%
#26-dropout rate=0.3-60s

In [ ]:
# 24 - 75%
# 26 - 36%
# 28 - 92%
# 30 - 4.1%
# 32 - 10%

# 26 - 38.6%  (1M)
# 28 - 91.7%  (1M)
# 30 - 11.69% (1M)
# 32 - 12.79% (1M)

# 32 - 2.3%   (1M)(Span=20)
# 32 - 1.7%   (1M)(Span=15)

# 32 (Train) - 38.6%   (train=1.2M)(Span=20)(epochs=10)
# 32 - 3.36%   (1M)(Span=20)
# 32 - 3.20%   (1M)(Span=20)
# 32 - 0.5%   (1M)(Span=20)
# 32 - 4.4%(10epoch),4.07%(50epoch)   (1M)(Span=20)

In [ ]:
# y_tr[0], np.argmax(y_pre[0]),y_pre[0]

In [ ]:
data = {'24':75.73,
        '26':38, '28':91, '30': 11, '32': 12}

# 20-100%
# 22-99.43%
# 24-99.73%
# 26-38.9%
#28-40.25%
# 30-44%
courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Stages in LC-RNG")
plt.ylabel("Accuracy")
# plt.title("Students enrolled in different courses")
plt.ylim(0,100)
plt.show()